In [1]:
from wrapper_functions import *
from ensemble import *

2022-11-07 11:29:33.156383: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/gpflow/experimental/utils.py:43: UserWarning: You're calling gpflow.experimental.check_shapes.decorator.check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  f"You're calling {name} which is considered *experimental*."
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/gpflow/experimental/utils.py:43: UserWarning: You're calling gpflow.experimental.check_shapes.inheritance.inherit_check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  f"You're calling {name} which i

TensorFlow version: 2.10.0. Expected: 2.7.0
TensorFlow Probability version: 0.18.0. Expected: 0.15.0


2022-11-07 11:30:02.342466: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
training_eastMA = pd.read_csv('../data/training_dataset/training_eastMA.csv')
training_eastMA_noMI = training_eastMA[:51]
training_eastMA_folds = pd.read_csv('../data/training_dataset/training_eastMA_folds.csv')
base_model_predictions_eastMA = pd.read_csv('../data/prediction_dataset/base_model_predictions_eastMA.csv')

training51= pd.read_csv('../data/training_dataset/training51.csv')
# standardize
X_train1 = np.asarray(training_eastMA_noMI[["lon", "lat"]].values.tolist()).astype(np.float32)
X_test1 = np.asarray(base_model_predictions_eastMA[["lon", "lat"]].values.tolist()).astype(np.float32)
X_valid = np.concatenate((X_train1, X_test1), axis=0)
X_centr = np.mean(X_valid, axis=0)
X_scale = np.max(X_valid, axis=0) - np.min(X_valid, axis=0)

X_train1 = (X_train1 - X_centr) / X_scale
X_test1 = (X_test1 - X_centr) / X_scale

Y_train = np.expand_dims(training_eastMA_noMI["aqs"], 1).astype(np.float32)
#Y_test = np.expand_dims(base_model_predictions_eastMA["pred_av"], 1).astype(np.float32)
base_model_names = ["pred_av", "pred_gs", "pred_caces"]
base_preds_train = tf.stack([training_eastMA_noMI[base_model_name].astype(np.float32) for base_model_name in base_model_names], axis=-1)
base_preds_test = tf.stack([base_model_predictions_eastMA[base_model_name].astype(np.float32) for base_model_name in base_model_names], axis=-1)
#base_preds_test

In [23]:
# Optimization configs. 
# Consider reduce below parameters / set to `False` if MCMC is taking too long:
# mcmc_num_steps, mcmc_burnin, mcmc_nchain, mcmc_initialize_from_map.
map_step_size=5e-4   # @param
map_num_steps=10_000  # @param

mcmc_step_size=1e-4 # @param
mcmc_num_steps=1000 # @param

mcmc_nchain=10 # @param
mcmc_burnin=2_500 # @param
bne_mcmc_initialize_from_map="True" # @param ["False", "True"]

bne_mcmc_initialize_from_map = eval(bne_mcmc_initialize_from_map)

# BMA parameters.
y_noise_std = 0.01  # Note: Changed from 0.1 # @param
bma_gp_lengthscale = .07 # @param
bma_gp_l2_regularizer = 0.15 # @param

bma_n_samples_train = 100 # @param
bma_n_samples_eval = 250 # @param
bma_n_samples_test = 250 # @param
bma_seed = 0 # @param

# BNE parameters.
bne_gp_lengthscale = .1 # 5. # @param
bne_gp_l2_regularizer = .05 # 15 # @param
bne_variance_prior_mean = -2.5 # @param
bne_skewness_prior_mean = -2.5 # @param
bne_seed = 0 # @param


In [66]:
# find the index of max and min  for lon and lat
min_index_lon = training_eastMA_noMI[['lon']].idxmin().values.tolist()
max_index_lon = training_eastMA_noMI[['lon']].idxmax().values.tolist()
min_index_lat = training_eastMA_noMI[['lat']].idxmin().values.tolist()
max_index_lat = training_eastMA_noMI[['lat']].idxmax().values.tolist()
# concetenate the index
edge_list = min_index_lon + max_index_lon + min_index_lat + max_index_lat
# exclude edge_list index from X_train1
train_wo_edge = training_eastMA_noMI[~np.isin(np.arange(len(X_train1)), edge_list)]
ref_model = LinearRegression()
kf = KFold(n_splits=10, shuffle=True, random_state=bma_seed)

rmse_lr = []
rmse_bma = []
rmse_gam = []

# concatenate train_wo_edge and edge_list index from training_eastMA_noMI
train_new_order = pd.concat([train_wo_edge, training_eastMA_noMI.iloc[edge_list]])
for train_index, test_index in kf.split(train_new_order[:-4]):
    train_index = train_index.tolist() + edge_list

    X_tr, X_te = X_train1[train_index], X_train1[test_index] 
    Y_tr, Y_te = Y_train[train_index], Y_train[test_index]

    base_preds_tr, base_preds_te = base_preds_train.numpy()[train_index], base_preds_train.numpy()[test_index]
    print(X_tr.shape, X_te.shape, Y_tr.shape, Y_te.shape, base_preds_tr.shape, base_preds_te.shape)
 
    # Ref: linear regression
    ref_model.fit(X_tr, Y_tr)
    Y_pred = ref_model.predict(X_te)
    rmse_lr.append(rmse(Y_te, Y_pred))
    #print(rmse_lr)

    #GMA
    ens_feature = np.asarray(list(base_preds_tr))
    term_list = [s(dim_index) for dim_index in range(ens_feature.shape[1])]
    term_list += [te(*list(ens_feature.shape[1] + np.array(range(X_tr.shape[1]))))]
    gam_feature_terms = TermList(*term_list)

    gam_X_tr = np.concatenate([X_tr, base_preds_tr], axis=1)
    gam_X_te = np.concatenate([X_te, base_preds_te], axis=1)
    ref_gam = LinearGAM(gam_feature_terms)
    #ref_gam = LinearGAM(te(0, 1, 2) + te(0, 1, 3) + te(0,1,4))
    gam = ref_gam.fit(gam_X_tr, Y_tr)
    Y_pred = gam.predict(gam_X_te)
    rmse_gam.append(rmse(Y_te, Y_pred))

    #BMA
   
    # build model & run MCMC
    bma_prior, bma_gp_config = bma_dist(X_tr, 
                                    base_preds_tr, 
                                    **bma_model_config)

    bma_model_config.update(bma_gp_config)


    bma_gp_w_samples = run_posterior_inference(model_dist=bma_prior, 
                                           model_config=bma_model_config,
                                           Y=Y_tr, 
                                           map_config=map_config,
                                           mcmc_config=mcmc_config)


    bma_joint_samples = make_bma_samples(X_te, None, base_preds_te, 
                                     bma_weight_samples=bma_gp_w_samples[0],
                                     bma_model_config=bma_model_config,
                                     n_samples=bma_n_samples_eval, 
                                     seed=bne_seed,
                                     y_samples_only=False)

    y_pred = bma_joint_samples['y']
    y_pred = tf.reduce_mean(y_pred, axis=0)

    rmse_bma.append(rmse(Y_te, y_pred))
    #print(rmse_bma)
print('RMSE of LR:', np.mean(rmse_lr)), print('RMSE of GAM:', np.mean(rmse_gam)), print('RMSE of BMA:', np.mean(rmse_bma))
print(rmse_lr, rmse_gam, rmse_bma)

(46, 2) (5, 2) (46, 1) (5, 1) (46, 3) (5, 3)


/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/pygam/pygam.py:752: RuntimeWarning: invalid value encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1) # invert the singular values
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


Running MAP:	336282.9375...120860.890625...113713.6171875...111760.953125...110947.265625...110511.515625...110242.6953125...109815.5078125...109720.703125...109670.6171875...Done.
Running MCMC:	Acceptance Ratio: 0.7041923403739929
(46, 2) (5, 2) (46, 1) (5, 1) (46, 3) (5, 3)


/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/pygam/pygam.py:752: RuntimeWarning: invalid value encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1) # invert the singular values
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


Running MAP:	357750.90625...125687.6875...118339.7265625...115935.859375...114805.7109375...114146.2734375...113697.9140625...113390.0625...113194.765625...113059.8359375...Done.
Running MCMC:	Acceptance Ratio: 0.7334452271461487
(46, 2) (5, 2) (46, 1) (5, 1) (46, 3) (5, 3)


...

Running MAP:	343334.125...120198.09375...111536.859375...109308.453125...108029.875...106516.5546875...105944.4609375...105637.6484375...105453.5390625...105331.2578125...Done.
Running MCMC:	Acceptance Ratio: 0.6871804594993591
(47, 2) (4, 2) (47, 1) (4, 1) (47, 3) (4, 3)


/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/pygam/pygam.py:593: RuntimeWarning: invalid value encountered in reciprocal
  weights ** -1)**-0.5)
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/pygam/pygam.py:752: RuntimeWarning: invalid value encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1) # invert the singular values
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/pygam/distributions.py:86: RuntimeWarning: invalid value encountered in reciprocal
  return (np.sum(weights * self.V(mu)**-1 * (y - mu)**2) /
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"

Running MAP:	360939.59375...125947.6796875...119117.3125...117293.6328125...116191.46875...115469.7890625...115094.7890625...114760.90625...113570.4609375...113348.484375...Done.
Running MCMC:	Acceptance Ratio: 0.7265361547470093
(47, 2) (4, 2) (47, 1) (4, 1) (47, 3) (4, 3)


/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/pygam/pygam.py:593: RuntimeWarning: invalid value encountered in reciprocal
  weights ** -1)**-0.5)
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/pygam/pygam.py:752: RuntimeWarning: invalid value encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1) # invert the singular values
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/pygam/distributions.py:86: RuntimeWarning: invalid value encountered in reciprocal
  return (np.sum(weights * self.V(mu)**-1 * (y - mu)**2) /
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer OrthogonalRandomFeatures is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"

Running MAP:	281118.96875...91320.1328125...84634.3984375...82532.3125...81156.125...80365.3125...79887.9765625...79521.1015625...79117.3203125...78809.46875...Done.
Running MCMC:	Acceptance Ratio: 0.6805007457733154
RMSE of LR: 1.5473506
RMSE of GAM: 1.624674532831662
RMSE of BMA: 1.2113247
[1.651134, 1.6793615, 1.1708562, 0.49377024, 1.7541255, 1.9771044, 1.2888193, 0.9464122, 1.0613754, 3.450548] [1.7644226193701453, 1.6390760208743562, 1.0855796719548323, 0.8788342904774931, 2.191957877670051, 1.9592359531327401, 0.931493983693236, 1.3288913068722057, 1.0827923323603172, 3.384461271911245] [1.1443169, 0.7335374, 0.59693074, 0.62986225, 1.859468, 1.3370078, 1.7261744, 1.3112383, 0.7378005, 2.03691]
